### Notebook Overview

| **Item**       | **Details**                         |
| -------------- | ----------------------------------- |
| **Author**  | Raphael Norris                         |
| **Date**    | 2025-09-19                          |
| **Purpose** | Notebook de exploracao de data quality|

In [2]:
import pandas as pd
import sys

sys.path.append('./../')
from src.data.conn_oracle import load_env_oracle
from src.data.conn_sql import *
import os
import pyarrow

In [13]:
df = query_sqlserver("SELECT TOP 10 * FROM dbo.Itens")
display(df)

✅ Conectado usando: ODBC Driver 17 for SQL Server


/home/suporte_amcom/Documentos/raphael-norris-ds/Projeto_IA_AMCOM/project_data_science/notebooks/../src/data/conn_sql.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,Item,Conjunto,idTipoFT,idTipoFT2,idTipoFT3,idTipoFT4,idTipoFT5,IDTipoIPI,IDClassFiscal,IDFamilia,...,AreaLiquidaPeca,AreaBrutaChapa,AreaLiquidaChapa,VolumePaleteFechadoM3,VolumePacoteFechadoM3,idUsuarioAnalise,PrecoFerramentas,QuantPrevista,ResumoChapa,ComplementoChapa
0,00001,00001,None,None,None,None,None,2,NaN,51,...,3.920000,3.920000,3.920000,NaN,NaN,133.0,None,1,-1549802772,144128
1,000020,00002,None,None,None,None,None,2,1229.0,9,...,0.476334,1.241840,1.208970,1.147500,0.060449,NaN,None,1,2145308917,5167364
2,000030,00003,None,None,None,None,None,2,1229.0,9,...,0.473293,1.508258,1.469853,1.355319,0.048995,NaN,None,1,1785235111,5167364
3,000040,00004,None,None,None,None,None,2,1229.0,9,...,0.320135,1.386720,1.350780,1.944000,0.033770,NaN,None,1,999763380,5167364
4,000042,00004,None,None,None,None,None,2,1229.0,9,...,0.052990,0.847385,0.828825,1.188000,0.005526,NaN,None,1,60748694,5167361
5,011310,01131,None,None,None,None,None,1,1229.0,20,...,1.039088,1.059668,1.052028,1.390327,0.073121,NaN,None,1,-212752887,5096452
6,100000,10000,None,None,None,None,None,2,1519.0,55,...,3.920000,3.920000,3.920000,NaN,NaN,NaN,None,1,-1381244890,6080258
7,100002,10000,None,None,None,None,None,2,1519.0,55,...,1.401400,1.401400,1.401400,NaN,NaN,NaN,None,1,-146609798,6080258
8,104200,10420,None,None,None,None,None,1,1229.0,51,...,0.687511,0.701256,0.701256,1.879200,NaN,NaN,None,1,1590685065,5104388
9,104202,10420,None,None,None,None,None,1,1567.0,51,...,0.118279,0.236558,0.236558,1.236749,NaN,NaN,None,1,1747149743,6341379


In [14]:
df.columns

Index(['Item', 'Conjunto', 'idTipoFT', 'idTipoFT2', 'idTipoFT3', 'idTipoFT4',
       'idTipoFT5', 'IDTipoIPI', 'IDClassFiscal', 'IDFamilia',
       ...
       'AreaLiquidaPeca', 'AreaBrutaChapa', 'AreaLiquidaChapa',
       'VolumePaleteFechadoM3', 'VolumePacoteFechadoM3', 'idUsuarioAnalise',
       'PrecoFerramentas', 'QuantPrevista', 'ResumoChapa', 'ComplementoChapa'],
      dtype='object', length=220)

In [16]:
df['DataCriacao']

0   2019-10-21 11:53:36.000
1   2014-01-09 08:16:17.477
2   2014-01-09 08:35:18.490
3   2014-01-09 09:19:05.733
4   2014-01-09 09:29:54.000
5   2025-04-10 14:20:46.423
6   2008-08-12 08:41:23.790
7   2008-08-12 08:48:38.610
8   2006-10-17 00:00:00.000
9   2007-09-25 16:43:30.720
Name: DataCriacao, dtype: datetime64[ns]

Clientes -> DataCriacao
Facas -> DataCriacaoRegistro
Cliches -> DataCriacaoRegistro
TarefCon ->DataCriacao
FI -> NA
Paradas -> NA
Maquina -> NA
Pedidos -> DataCriacao
Itens ->DataCriacao


  


In [ ]:
from Projeto_IA_AMCOM.project_data_science.src.data.data_quality_analytics import analyze_for_exclusion

tables_with_date = {
    # "Clientes": "DataCriacao",
    # "Facas": "DataCriacaoRegistro",
    # "Cliches": "DataCriacaoRegistro",
    "TarefCon": "DataCriacao",
    "FI": None,
    "Paradas": None,
    "Maquina": None,
    "Pedidos": "DataCriacao",
    "Itens": "DataCriacao",
}

date_filter_value = "2022-01-01"

results = {}

for table, date_column in tables_with_date.items():
    if date_column:  
        filters = {
            date_column: {"operator": ">=", "value": date_filter_value}
        }
    else:
        filters = {}  

    result = analyze_for_exclusion(
        table_name=table,
        schema="dbo",
        strict=False,
        filters=filters
    )

    results[table] = result

result



IDENTIFICANDO COLUNAS PARA EXCLUSÃO
Tabela: dbo.TarefCon
Filtros aplicados:
  - DataCriacao >= 2022-01-01
Critérios: Nulos >90%, Valor Único (=1), Zeros >80%
Query executada: SELECT * FROM dbo.TarefCon WHERE DataCriacao >= ?
Parâmetros: ('2022-01-01',)
Tentando pymssql...
pymssql falhou: module 'pymssql' has no attribute 'connect'
Tentando pyodbc com ODBC Driver 17 for SQL Server...
SUCESSO: Conectado via pyodbc + ODBC Driver 17 for SQL Server
Analisando 1,604,727 registros, 44 colunas

ANALISE DETALHADA DE TODAS AS COLUNAS:
----------------------------------------------------------------------
Maquina                   MANTER   - 34 únicos (0.0%), 0.0% nulos
Tarefa                    MANTER   - 1604727 únicos (100.0%), 0.0% nulos
FlagParada                MANTER   - 2 únicos (0.0%), 0.0% nulos, 19.9% zeros
CodigoParadaOuConv        MANTER   - 39 únicos (0.0%), 19.9% nulos, 0.9% zeros
Turma                     MANTER   - 3 únicos (0.0%), 0.0% nulos
OP                        MANTER   -

{'total_columns': 220,
 'columns_to_exclude': ['idTipoFT',
  'idTipoFT2',
  'idTipoFT3',
  'idTipoFT4',
  'idTipoFT5',
  'IDAgrupamento',
  'RazaoSusp',
  'CodigoEAN',
  'CrushMinimo',
  'CobbInt30Maximo',
  'CobbExt30Maximo',
  'TesteFisicoAdicional1',
  'PAT_Interno',
  'PAT_Externo',
  'Umidade',
  'Compressao',
  'ProfundidadeVinco',
  'ForcaDeVinco1',
  'ForcaDeVinco2',
  'ForcaDeVinco3',
  'FormatoSimplex',
  'VincosLargNoRiscador',
  'VincoLarg4',
  'VincoLarg5',
  'VincoLarg6',
  'VincoLarg7',
  'VincoLarg8',
  'VincoLarg9',
  'VincoLarg10',
  'VincosCompNoRiscador',
  'VincoComp6',
  'VincoComp7',
  'VincoComp8',
  'VincoComp9',
  'VincoComp10',
  'TearTape1',
  'TearTape2',
  'TearTape3',
  'TearTape4',
  'ProlongLap',
  'Rejeito',
  'RejeitoPecas',
  'ResinaInterna',
  'ResinaInternaSN',
  'ResinaExterna',
  'ResinaExternaSN',
  'EndurecedorMiolo',
  'EndurecedorMioloSN',
  'Grampos',
  'FitilhosCompPacote',
  'FacesOpostas',
  'Cantoneiras',
  'OrelhasInvertidas',
  'Consum

In [ ]:
from Projeto_IA_AMCOM.project_data_science.src.data.data_quality_analytics import analyze_for_exclusion
Clientes -> DataCriacao
Facas -> DataCriacaoRegistro
Cliches -> DataCriacaoRegistro
TarefCon ->DataCriacao
FI -> NA
Paradas -> NA
Maquina -> NA
Pedidos -> DataCriacao
Itens ->DataCriacao

filters = {
    'DataCriacaoRegistro': {'operator': '>=', 'value': '2022-01-01'}
}

# Análise para exclusão
result = analyze_for_exclusion(
    table_name='Cliches', 
    schema='dbo', 
    strict=False,
    filters=filters
)
        


IDENTIFICANDO COLUNAS PARA EXCLUSÃO
Tabela: dbo.Cliches
Filtros aplicados:
  - DataCriacaoRegistro >= 2022-01-01
Critérios: Nulos >90%, Valor Único (=1), Zeros >80%
Query executada: SELECT * FROM dbo.Cliches WHERE DataCriacaoRegistro >= ?
Parâmetros: ('2022-01-01',)
Tentando pymssql...
pymssql falhou: module 'pymssql' has no attribute 'connect'
Tentando pyodbc com ODBC Driver 17 for SQL Server...
SUCESSO: Conectado via pyodbc + ODBC Driver 17 for SQL Server
Analisando 16,786 registros, 17 colunas

ANALISE DETALHADA DE TODAS AS COLUNAS:
----------------------------------------------------------------------
CodCliche                 MANTER   - 16786 únicos (100.0%), 0.0% nulos
Tipo                      EXCLUIR  - VALOR ÚNICO (1)
DescrTipoCliche           EXCLUIR  - VALOR ÚNICO (Cyrel)
Fornecedor                MANTER   - 69 únicos (0.6%), 26.2% nulos, 0.2% vazias
Modulo                    MANTER   - 56 únicos (0.7%), 51.3% nulos, 0.1% vazias
Arquivo                   EXCLUIR  - MUITOS N

In [20]:
null_percentages = (df.isnull().sum() / len(df)) * 100

high_null_columns = null_percentages[null_percentages > 50]

print(f"\nColunas com mais de 50% de valores nulos: {len(high_null_columns)}")
print("\nDetalhamento:")

if len(high_null_columns) > 0:
    for col, pct in high_null_columns.sort_values(ascending=False).items():
        print(f"  {col}: {pct:.1f}% nulls")
else:
    print("  Nenhuma coluna com mais de 50% de valores nulos")

print(f"\nResumo:")
print(f"- Colunas com 0% nulls: {len(null_percentages[null_percentages == 0])}")
print(f"- Colunas com 1-25% nulls: {len(null_percentages[(null_percentages > 0) & (null_percentages <= 25)])}")
print(f"- Colunas com 26-50% nulls: {len(null_percentages[(null_percentages > 25) & (null_percentages <= 50)])}")
print(f"- Colunas com 51-75% nulls: {len(null_percentages[(null_percentages > 50) & (null_percentages <= 75)])}")
print(f"- Colunas com 76-100% nulls: {len(null_percentages[null_percentages > 75])}")


Colunas com mais de 50% de valores nulos: 24

Detalhamento:
  CodigoERP: 100.0% nulls
  CartaOrcamento: 100.0% nulls
  PacotesAltura: 100.0% nulls
  UsoGeralStr1: 100.0% nulls
  UsoGeralStr2: 100.0% nulls
  IDClassFiscal: 100.0% nulls
  ContatoFaturamento: 100.0% nulls
  idAtendenteVenda: 100.0% nulls
  FitasLargPalete: 100.0% nulls
  PressaoDeArqueamento: 100.0% nulls
  FitasCompPalete: 100.0% nulls
  IDPalete: 100.0% nulls
  PacotesComprimento: 100.0% nulls
  PacotesLargura: 100.0% nulls
  RazaoTravaComposicao: 100.0% nulls
  UsuarioCriacao: 99.9% nulls
  UsoGeralNum2: 99.9% nulls
  Agrupamento: 99.7% nulls
  TipoABC: 99.7% nulls
  Etiqueta: 98.4% nulls
  Laudo: 98.1% nulls
  EmailLaudo: 97.9% nulls
  UsuarioUltModif: 97.3% nulls
  CodSegmento: 77.7% nulls

Resumo:
- Colunas com 0% nulls: 37
- Colunas com 1-25% nulls: 9
- Colunas com 26-50% nulls: 0
- Colunas com 51-75% nulls: 0
- Colunas com 76-100% nulls: 24


In [25]:
df =df.drop(columns = null_percentages[null_percentages>50].index)

In [27]:
df.describe()

,IDCliente,CodRepresentante,TolMais,TolMenos,Cliente_Fornecedor,NrEtiquetas,AceitaPaleteIncompleto,DataCriacao,DataUltModif,CustoFinanceiroAdicional,...,CustoAdicTransportePorPeca_Paletizado,CustoAdicTransportePorPeca_NaoPaletizado,UsoGeralNum1,Paletizado,FacesOpostas,Espelho,Cantoneiras,Filme,OrelhasInvertidas,EmpilhamentoMaximo
count,5586.000000,5586.000000,5586.000000,5586.000000,5586.0,5585.00000,5586.000000,5586,5586,5586.0,...,5586.0,5586.0,4791.0,5586.000000,5586.0,5586.000000,5586.0,5586.000000,5586.0,5586.0
mean,9637.146079,49.241139,5.624776,5.631042,0.0,0.00752,1.009667,2019-04-05 14:37:24.809198080,2025-05-15 22:46:19.690691328,0.0,...,0.0,0.0,0.0,0.000179,0.0,0.003580,0.0,0.000179,0.0,1.0
min,546.000000,3.000000,0.000000,0.000000,0.0,0.00000,1.000000,2018-01-23 09:42:32.883000,2025-04-09 12:10:37.240000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0
25%,6367.250000,9.000000,0.000000,0.000000,0.0,0.00000,1.000000,2018-01-23 09:42:32.883000064,2025-05-05 16:40:25.492999936,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0
50%,9906.500000,18.000000,10.000000,10.000000,0.0,0.00000,1.000000,2018-01-23 09:42:32.883000064,2025-05-05 16:40:25.492999936,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0
75%,12846.750000,90.000000,10.000000,10.000000,0.0,0.00000,1.000000,2019-02-05 13:58:45.955750144,2025-05-05 16:40:25.492999936,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0
max,20470.000000,213.000000,10.000000,10.000000,0.0,1.00000,3.000000,2025-09-23 09:50:35.390000,2025-09-23 09:50:35.400000,0.0,...,0.0,0.0,0.0,1.000000,0.0,3.000000,0.0,1.000000,0.0,1.0
std,4823.414814,57.052025,4.822870,4.820327,0.0,0.08640,0.103197,NaN,NaN,0.0,...,0.0,0.0,0.0,0.013380,0.0,0.086645,0.0,0.013380,0.0,0.0


In [26]:
df.head()

,IDCliente,Cliente,CodCliente,CodRepresentante,TolMais,TolMenos,TipoObs1,Obs1,TipoObs2,Obs2,...,ExigeAgendamentoEntregas,ClienteProspecto,ImprimeLogotipo,Paletizado,FacesOpostas,Espelho,Cantoneiras,Filme,OrelhasInvertidas,EmpilhamentoMaximo
0,546,SADIA - PARANAGUA,10561,90,5,5,Expedição,NAO MANDAR CARRETA REBAIXADA,Expedição,NAO EMBARCAR MAIS QUE 5% DO PEDIDO,...,False,False,True,0,0,0,0,0,0,1
1,547,CRUZADO IND. PROD. DE LIMP. LTDA,9330,11,10,10,Paletização,"ALTURA MÁXIMA DRIVE IN 1,30",Conversão,"ALTURA PALETE MÁXIMO 1,300",...,False,False,True,0,0,0,0,0,0,1
2,1144,AIDA ALIMENTOS LTDA,10015,18,10,10,Expedição,OBRIGATORIO ENVIAR CHAPAS PARA DESCARREGAR CAM...,Expedição,RECEBIMENTO DE SEGUNDA A QUINTA - PREFERENCIAL...,...,False,False,True,0,0,0,0,0,0,1
3,1145,LATCO - F.BELTRAO,10021,3,10,10,,,,,...,False,False,True,0,0,0,0,0,0,1
4,1146,LATCO BEVERAGES IND.ALIMENTOS LTDA.-,10032,3,10,10,,,,,...,False,False,True,0,0,0,0,0,0,1


In [ ]:
import pymssql
print("Atributos disponíveis:", dir(pymssql))
print("Tem connect?", hasattr(pymssql, 'connect'))
print("Localização:", pymssql.__file__)

Atributos disponíveis: ['ORACLE_HOST', 'ORACLE_PORT', 'ORACLE_RAW_PASSWORD', 'ORACLE_RAW_USER', 'ORACLE_REFINED_PASSWORD', 'ORACLE_REFINED_USER', 'ORACLE_SERVICE_NAME', 'ORACLE_TRUSTED_PASSWORD', 'ORACLE_TRUSTED_USER', 'SQLSERVER_DATABASE', 'SQLSERVER_HOST', 'SQLSERVER_PASSWORD', 'SQLSERVER_USER', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'create_engine', 'get_connection', 'get_oracle_connection', 'get_oracle_engine', 'get_sqlserver_connection', 'get_sqlserver_engine', 'load_dotenv', 'os', 'pd', 'pymssql', 'query_oracle', 'query_oracle_raw', 'query_oracle_refined', 'query_oracle_trusted', 'query_sqlserver', 'query_to_dataframe', 'warnings']
Tem connect? False
Localização: /home/suporte_amcom/Documentos/raphael-norris-ds/.venv/lib/python3.12/site-packages/pymssql/__init__.py
